In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
cols = ['id_hogar', 'id_persona', 'p4_edad', 'p5_id_nivel_educativo', 'p6_id_ocupacion', 'p13v_condujo_bicicleta', 'f_exp', 'Sexo']
df_hogares = pd.read_csv('../data/csv/HogaresEODH_2019.csv', sep=';')
df_personas = pd.read_csv('../data/csv/PersonasEODH_2019.csv', sep=';', usecols=cols)

In [ ]:
df_hogares['vehiculo'] = df_hogares[['p1mc_automovil', 'p1mc_camionetas', 'p1mc_pickup', 'p1mc_motocicleta']].sum(axis=1).astype(bool)

In [ ]:
df = df_personas.merge(df_hogares[['Id_Hogar', 'vehiculo']], left_on='id_hogar', right_on='Id_Hogar').drop(['Id_Hogar'], axis=1)

In [ ]:
df['p13v_condujo_bicicleta'] = df['p13v_condujo_bicicleta'].map({1: True, 2: False})

In [ ]:
est_cod = list(range(1, 6))
est_dicc = dict.fromkeys(est_cod, 'Estudia')

trab_cod = list(range(11, 22))
trab_dicc = dict.fromkeys(trab_cod, 'Trabaja')

otro_cod = list(range(31, 39))
otro_dicc = dict.fromkeys(otro_cod, 'Otro')

d = {**est_dicc, **trab_dicc, **otro_dicc}

dicc_nivel_ed = {
    1: 'Primaria',
    2: 'Primaria',
    3: 'Primaria',
    4: 'Primaria',
    5: 'Primaria',
    6: 'Primaria',
    7: 'Media',
    8: 'Media',
    9: 'Superior',
    10: 'Media',
    11: 'Superior',
    12: 'Superior',
    13: 'Posgrado',
    14: 'Ninguno'
}

In [ ]:
df['p6_id_ocupacion'] = df['p6_id_ocupacion'].map(d)
df['p5_id_nivel_educativo'] = df['p5_id_nivel_educativo'].map(dicc_nivel_ed)

In [ ]:
df = df.drop(['id_hogar', 'id_persona', 'f_exp'], axis=1).dropna()

In [ ]:
df = pd.concat([df, pd.get_dummies(df['p6_id_ocupacion'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['p5_id_nivel_educativo'])], axis=1)

In [ ]:
df = df.drop(['p5_id_nivel_educativo', 'p6_id_ocupacion'], axis=1)

In [ ]:
df['Sexo'] = df['Sexo'].map({'Mujer': 0, 'Hombre': 1})

In [ ]:
df['vehiculo'] = df['vehiculo'].astype(int)
df['p13v_condujo_bicicleta'] = df['p13v_condujo_bicicleta'].astype(int)

In [ ]:
# count = df['p13v_condujo_bicicleta'].sum()
# remove_n = df.query('p13v_condujo_bicicleta == 0').shape[0] - count

In [ ]:
# drop_indices = np.random.choice(df.query('p13v_condujo_bicicleta == 0').index, remove_n, replace=False)
# df = df.drop(drop_indices)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['p13v_condujo_bicicleta'], axis=1), df['p13v_condujo_bicicleta'], random_state=42)

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
result = clf.predict(X_test)

In [ ]:
(result == y_test.values).sum()

In [ ]:
(result == y_test.values)

In [ ]:
probas = clf.predict_proba(df.drop(['p13v_condujo_bicicleta'], axis=1))

In [ ]:
probas

In [ ]:
df['proba'] = probas[..., 1]

In [ ]:
df

In [ ]:
X_test['prediccion'] = result
X_test['real'] = y_test